In [2]:
%load_ext autoreload
%autoreload 2
import os
import codecs
from metal.mmtl.debugging.slicing import *
from metal.mmtl.glue.glue_preprocess import *
from metal.mmtl.eval_slices import *

## Slices

In [5]:
task_name = 'MRPC'
split = 'dev'
func_name = 'lowbleu'

slice_fn = globals()['slice_'+func_name]
tsv_path = f'/dfs/scratch0/bradenjh/glue/{task_name}/{split}.tsv'

In [6]:
config = get_task_tsv_config(task_name,split)

sent1_idx = config["sent1_idx"]
sent2_idx = config["sent2_idx"]
filtered_rows = []
all_rows = []
with codecs.open(tsv_path, "r", "utf-8") as data_fh:
    if config["skip_rows"] > 0:
        header = data_fh.readline()
        filtered_rows.append(header)
        all_rows.append(header)
    for i, row in enumerate(data_fh):
        sent1 = row.split('\t')[sent1_idx]
        sent2 = row.split('\t')[sent2_idx]
        in_slice = slice_fn({'sentence1': sent1, 'sentence2': sent2})
        if in_slice:
            filtered_rows.append(row)
        all_rows.append(row)
print(f'slice prop: {len(filtered_rows)}/{len(all_rows)}')

slice prop: 430/1726


In [8]:
type(all_rows)

list

In [24]:
model_path = '/dfs/scratch0/chami/metal/metal/mmtl/aws/output/2019_03_14_01_58_14/1/logdir/bert_large/QNLI.STSB.MRPC.QQP.RTE.MNLI.SST2.COLA.WNLI_09_16_00'
if 'best_model.pth' in model_path:
    model_path = model_path.replace('/best_model.pth','')
task_names = [task_name]
slice_names = [func_name]
slice_performance = eval_on_slices(model_path, task_names, slice_names,'dev')


Could not find kwarg "generate_uids" in destination dict.
Using random seed: 582664



  6%|▌         | 76459008/1248501532 [00:09<08:54, 2191275.72B/s]

KeyboardInterrupt: 

In [89]:
with codecs.open(save_path, "w", "utf-8") as data_fh:
    for row in filtered_rows:
        data_fh.write(row)